In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.io
from scipy import stats
from scipy.linalg import lstsq
from sklearn import decomposition
from lmfit import Model, Parameters
import math

ModuleNotFoundError: No module named 'lmfit'

In [ ]:
path = Path.home()/'code'/'2023_CENTURI-SummerSchool'/'datasets'/'dataset1_reaching-task'/'Dataset1.mat'
data = scipy.io.loadmat(path)
R = data['R']
direction = data['direction']

In [ ]:
print(R.shape) # 143 neurons x 158 trials
print(direction.shape) # 1-8x

In [ ]:
R_mean = R.mean(axis=1)

In [ ]:
fig, ax = plt.subplots(nrows=2,ncols=1,figsize=(16,8))

ax[0].bar(range(158),direction[:,0])
ax[0].set_ylabel('movement direction')

ax[1].plot(R[0,:])
ax[1].set_xlabel('trials')
ax[1].set_ylabel('firing rate of neuron 0')

In [ ]:
for dir_val in range(1,9):
    indices = np.where(direction[:,0]==dir_val)[0]
    print(f'{direction[indices,0].shape[0]} trials for direction {dir_val}')

In [ ]:
# Mean firing rate for neuron 0 along all directions
mean_dir_FR = []
std_dir_FR = []

for dir_val in range(1,9):
    indices = np.where(direction[:,0]==dir_val)[0]
    mean_dir_FR.append(R[0,indices].mean())
    std_dir_FR.append(R[0,indices].std())
plt.errorbar(range(8),mean_dir_FR,yerr=std_dir_FR)
plt.xticks(ticks=range(8),labels=range(1,9))
plt.title('neuron 0')
plt.ylabel('mean firing rate over trials')
plt.xlabel('direction')
plt.show()

In [ ]:
def bootstrapping(firing_rates,num_iterations=1000):
    # Perform bootstrapping
    bootstrapped_means = []
    for _ in range(num_iterations):
        resample = np.random.choice(firing_rates, size=len(firing_rates), replace=True)
        bootstrapped_mean = np.mean(resample)
        bootstrapped_means.append(bootstrapped_mean)
    # return np.std(bootstrapped_means)
    return np.quantile(bootstrapped_means,q=[0.05,0.95])

In [ ]:
# Mean firing rate for all neuron along all directions
fig, ax = plt.subplots(nrows=3,ncols=8,figsize=(25,8))
for i in range(8):
    mean_dir_FR = []
    std_dir_FR = []
    standard_errors_per_trial = []
    bootstrapping_stds_per_trial = []
    for dir_val in range(1,9):
        indices = np.where(direction[:,0]==dir_val)[0]
        mean_dir_FR.append(R[i,indices].mean())
        std_dir_FR.append(R[i,indices].std())
        standard_errors_per_trial.append(stats.sem(R[i,indices])) # Calculating standard error
        bootstrapping_stds_per_trial.append(bootstrapping(R[i,indices]))
        
    ax[0][i].errorbar(range(8),mean_dir_FR,yerr=std_dir_FR)
    ax[0][i].set_ylabel('mean firing rate over trials')
    ax[0][i].set_xlabel('direction')
    
    ax[1][i].plot(standard_errors_per_trial)
    ax[1][i].set_ylabel('standard error')
    
    ax[2][i].plot(bootstrapping_stds_per_trial)
    ax[2][i].set_ylabel('bootstrapping std')
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=10,subplot_kw={'projection': 'polar'},figsize=(20,8))

for i in range(10):
    mean_dir_FR = []
    for dir_val in range(1,9):
        indices = np.where(direction[:,0]==dir_val)[0]
        mean_dir_FR.append(R[i,indices].mean())

    # angles = np.arange(8)
    angles = np.linspace(0, 2 * np.pi, 8)
    responses = np.cos(2 * angles)

    ax[i].scatter(angles, mean_dir_FR)

    # ax.set_rticks([-1, 0, 1])  # Set radial ticks
    ax[i].set_rlabel_position(180)  # Label position along radial axes
    ax[i].grid(True)  # Add a grid

    ax[i].set_title(i)
plt.tight_layout()
plt.show()

In [ ]:
# https://gist.github.com/laurentperrinet/de76f53fcb0820844fbf0317ed832035

# https://en.wikipedia.org/wiki/Von_Mises_distribution
def tuning_function(theta, theta0, kappa, fmax, bsl, theta_bound):
    # Von Mises, with kappa the concentration, theta0 the location
    # fmax the firing rate at pref ori, bsl the min firing rate (not the baseline, which was substracted) 
    tf = bsl + np.exp(kappa*(np.cos(2*np.pi/theta_bound*(theta-theta0))-1)) * (fmax-bsl)
    return tf

def fit_tc(array, init_kappa, theta_bound=2*np.pi):
    """
    Fits the data points in `array` to `tuning_function`.
    
    set theta_bound to 2*np.pi for fitting signed angles (eg dirrection)
    or to np.pi for an unsigned one (like orientation)
    """
              
    theta = np.linspace(0, theta_bound, len(array), endpoint=False)
    mod = Model(tuning_function)
    pars = Parameters()
    #               name    default           vary  min   max
    pars.add_many(('theta0', theta[np.argmax(array)], True, 0., theta_bound),
                  ('kappa', init_kappa, True,  .1, 5.),
                  ('fmax', np.max(array), True, 0.0, 2*np.max(array)+5),
                  ('bsl', np.min(array), True, 0.0, np.max(array)+5),
                  ('theta_bound', theta_bound, False))
                
    out = mod.fit(array, pars, theta=theta, nan_policy='omit', max_nfev=50000)

    return out.best_values

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=10,figsize=(20, 4))

for i in range(10):
    mean_dir_FR = []
    for dir_val in range(1,9):
        indices = np.where(direction[:,0]==dir_val)[0]
        mean_dir_FR.append(R[i,indices].mean())
    fitted_params = fit_tc(np.array(mean_dir_FR), 3.5,theta_bound=8)
    print(f"neuron {i} theta0 = {fitted_params['theta0']} kappa (precision) = {fitted_params['kappa']}")
    theta_more = np.linspace(0, 8, 60, endpoint=True)
    ax[i].plot(mean_dir_FR)
    tf = tuning_function(theta_more, **fitted_params)
    ax[i].plot(theta_more, tf)
    ax[i].set_xlabel('direction')
    ax[i].set_ylabel('mean FR')
plt.tight_layout()

In [ ]:
# Checking the tuning precision for all the neurons

high_kappa_neuron_ids = []
kappa_threshold = 4.5

kappas = []
theta0s = []

for i in range(R.shape[0]):
    mean_dir_FR = []
    for dir_val in range(1,9):
        indices = np.where(direction[:,0]==dir_val)[0]
        mean_dir_FR.append(R[i,indices].mean())
    fitted_params = fit_tc(np.array(mean_dir_FR), 3.5,theta_bound=8)
    kappas.append(fitted_params['kappa'])
    theta0s.append(fitted_params['theta0'])
    # print(f"neuron {i} theta0 = {fitted_params['theta0']} kappa (precision) = {fitted_params['kappa']}")
    if fitted_params['kappa'] >= kappa_threshold: high_kappa_neuron_ids.append(i)
plt.hist(kappas,bins=20)
plt.title('tuning precision for all the neurons')
plt.xlabel('Von Mises kappa (precision)')
plt.ylabel('number of neurons')

print(high_kappa_neuron_ids)

In [ ]:
max_inds = np.argsort(np.array(kappas)[high_kappa_neuron_ids])
max_real_inds = np.array(high_kappa_neuron_ids)[max_inds]
max_five_inds = np.flip(max_real_inds[-5:])
max_five_inds

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=5,figsize=(20, 4))

for i in range(5):
    mean_dir_FR = []
    for dir_val in range(1,9):
        indices = np.where(direction[:,0]==dir_val)[0]
        mean_dir_FR.append(R[max_five_inds[i],indices].mean())
    fitted_params = fit_tc(np.array(mean_dir_FR), 3.5,theta_bound=8)
    theta_more = np.linspace(0, 8, 60, endpoint=True)
    ax[i].plot(mean_dir_FR)
    tf = tuning_function(theta_more, **fitted_params)
    # print(tuning_function(, **fitted_params))
    ax[i].plot(theta_more, tf)
    ax[i].set_xlabel('direction')
    ax[i].set_ylabel('mean FR')
plt.suptitle('Five neurons with highest kappa')
plt.tight_layout()

In [ ]:
plt.scatter(kappas, R_mean)
plt.xlabel('kappa')
plt.ylabel('mean FR')

Tuning depth = maximum FR - minimum FR

Outliers would have low tuning depth

In [ ]:
plt.hist(theta0s,bins=20)
plt.title('direction preference of all neurons')
plt.xlabel('preferred direction (theta0)')
plt.ylabel('number of neurons')

In [ ]:
mask = np.ones(np.array(theta0s).shape, dtype=bool)
mask[high_kappa_neuron_ids] = False
theta0s_without_high_kappa = np.array(theta0s)[mask]

plt.hist(theta0s,bins=20)
plt.title('direction preference')
plt.xlabel('preferred direction (theta0)')
plt.ylabel('number of neurons')

plt.hist(theta0s_without_high_kappa,bins=20)
plt.xlabel('preferred direction (theta0)')
plt.ylabel('number of neurons')

plt.legend(['all neurons',f'only neurons with kappa < {kappa_threshold}'])

Overrepresentation of certain directions - related to geometry of the arm

In [ ]:
for dir_val in range(1,9):
    indices = np.where(direction[:,0]==dir_val)[0]
    plt.scatter(R[2,indices],R[3,indices])
plt.xlabel('neuron 0 FR')
plt.ylabel('neuron 1 FR')
plt.legend(range(1,9))

In [ ]:
plt.title('Poisson statistics: variability is proportional to average firing rate')
plt.scatter(R.mean(axis=0),R.std(axis=0))
plt.xlabel('FR mean')
plt.ylabel('FR std')

In [ ]:
# Center the data (subtract mean)
R_centered = R.copy()
for i in range(R.shape[0]):
    R_centered[i,:] = R_centered[i,:] - R_centered[i,:].mean() 

In [ ]:
pca = decomposition.PCA(n_components=8)
x_pca = pca.fit_transform(R_centered)
x_pca.shape

In [ ]:
plt.scatter(x_pca[:,0], x_pca[:,1])
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('PCA on the full R matrix (centered)')

In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_

In [ ]:
x_range = [x for x in range(0, len(explained_variance_ratio))]
x_range_labels = [x+1 for x in range(0, len(explained_variance_ratio))]
plt.plot(x_range, explained_variance_ratio, marker='o')
plt.xticks(ticks=x_range,labels=x_range_labels)
plt.xlabel('Number of Principal Component')
plt.ylabel('Explained Variance')
plt.title('Scree Plot')
plt.show()

In [ ]:
R_conds = np.zeros((R.shape[0], 8))

for dir_val in range(1,9):
    indices = np.where(direction[:,0]==dir_val)[0]
    R_conds[:,dir_val-1] = R_centered[:,indices].mean(axis=1)
R_conds.shape

In [ ]:
pca2 = decomposition.PCA(n_components=2)
x_pca2 = pca2.fit_transform(R_conds)
x_pca2.shape

In [ ]:
plt.scatter(x_pca2[:,0], x_pca2[:,1])
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('PCA on the R matrix with trials averaged over directions [neurons x directions]')

In [ ]:
for dir_val in range(1,9):
    indices = np.where(direction[:,0]==dir_val)[0]
    pc_space_points = np.dot(R_centered[:,indices].T, x_pca2[:,:])
    plt.scatter(pc_space_points[:,0], pc_space_points[:,1])
    
plt.legend(range(1,9))
plt.title('PCA projected R matrix')

In [ ]:
pc_space_points = np.dot(R_centered[:,:].T, x_pca2[:,:])
plt.scatter(pc_space_points[:,0],pc_space_points[:,1])
plt.title('PCA projected R matrix')

In [ ]:
pc_space_points_centered = pc_space_points.copy()
pc_space_points_centered = pc_space_points_centered - pc_space_points_centered.mean()

x = pc_space_points_centered[:,0]
y = pc_space_points_centered[:,1]

# Construct the matrix A
A = np.vstack([2 * x, 2 * y, np.ones(len(x))]).T

# Formulate the linear system Ax = b
b = x**2 + y**2

# Solve the linear system using least squares
x_coope, _, _, _ = lstsq(A, b)

# Extract the parameters of the fitted circle
xc = x_coope[0]
yc = x_coope[1]
r = np.sqrt(x_coope[2] + xc**2 + yc**2)

print("Fitted circle parameters:")
print("Center (xc, yc):", xc, yc)
print("Radius (r):", r)


In [ ]:
pc_space_points.shape

In [ ]:
plt.scatter(pc_space_points_centered[:,0],pc_space_points_centered[:,1])
circle = plt.Circle((xc, yc), r, color='red', fill=False, label='Fitted Circle')
plt.gca().add_patch(circle)
plt.title('fitting a circle to data in PCA space')

In [ ]:
def fit_ellipse(x, y):
    """

    Fit the coefficients a,b,c,d,e,f, representing an ellipse described by
    the formula F(x,y) = ax^2 + bxy + cy^2 + dx + ey + f = 0 to the provided
    arrays of data points x=[x1, x2, ..., xn] and y=[y1, y2, ..., yn].

    Based on the algorithm of Halir and Flusser, "Numerically stable direct
    least squares fitting of ellipses'.


    """

    D1 = np.vstack([x**2, x*y, y**2]).T
    D2 = np.vstack([x, y, np.ones(len(x))]).T
    S1 = D1.T @ D1
    S2 = D1.T @ D2
    S3 = D2.T @ D2
    T = -np.linalg.inv(S3) @ S2.T
    M = S1 + S2 @ T
    C = np.array(((0, 0, 2), (0, -1, 0), (2, 0, 0)), dtype=float)
    M = np.linalg.inv(C) @ M
    eigval, eigvec = np.linalg.eig(M)
    con = 4 * eigvec[0]* eigvec[2] - eigvec[1]**2
    ak = eigvec[:, np.nonzero(con > 0)[0]]
    return np.concatenate((ak, T @ ak)).ravel()


def cart_to_pol(coeffs):
    """

    Convert the cartesian conic coefficients, (a, b, c, d, e, f), to the
    ellipse parameters, where F(x, y) = ax^2 + bxy + cy^2 + dx + ey + f = 0.
    The returned parameters are x0, y0, ap, bp, e, phi, where (x0, y0) is the
    ellipse centre; (ap, bp) are the semi-major and semi-minor axes,
    respectively; e is the eccentricity; and phi is the rotation of the semi-
    major axis from the x-axis.

    """

    # We use the formulas from https://mathworld.wolfram.com/Ellipse.html
    # which assumes a cartesian form ax^2 + 2bxy + cy^2 + 2dx + 2fy + g = 0.
    # Therefore, rename and scale b, d and f appropriately.
    a = coeffs[0]
    b = coeffs[1] / 2
    c = coeffs[2]
    d = coeffs[3] / 2
    f = coeffs[4] / 2
    g = coeffs[5]

    den = b**2 - a*c
    if den > 0:
        raise ValueError('coeffs do not represent an ellipse: b^2 - 4ac must'
                         ' be negative!')

    # The location of the ellipse centre.
    x0, y0 = (c*d - b*f) / den, (a*f - b*d) / den

    num = 2 * (a*f**2 + c*d**2 + g*b**2 - 2*b*d*f - a*c*g)
    fac = np.sqrt((a - c)**2 + 4*b**2)
    # The semi-major and semi-minor axis lengths (these are not sorted).
    ap = np.sqrt(num / den / (fac - a - c))
    bp = np.sqrt(num / den / (-fac - a - c))

    # Sort the semi-major and semi-minor axis lengths but keep track of
    # the original relative magnitudes of width and height.
    width_gt_height = True
    if ap < bp:
        width_gt_height = False
        ap, bp = bp, ap

    # The eccentricity.
    r = (bp/ap)**2
    if r > 1:
        r = 1/r
    e = np.sqrt(1 - r)

    # The angle of anticlockwise rotation of the major-axis from x-axis.
    if b == 0:
        phi = 0 if a < c else np.pi/2
    else:
        phi = np.arctan((2.*b) / (a - c)) / 2
        if a > c:
            phi += np.pi/2
    if not width_gt_height:
        # Ensure that phi is the angle to rotate to the semi-major axis.
        phi += np.pi/2
    phi = phi % np.pi

    return x0, y0, ap, bp, e, phi


def get_ellipse_pts(params, npts=100, tmin=0, tmax=2*np.pi):
    """
    Return npts points on the ellipse described by the params = x0, y0, ap,
    bp, e, phi for values of the parametric variable t between tmin and tmax.

    """

    x0, y0, ap, bp, e, phi = params
    # A grid of the parametric variable, t.
    t = np.linspace(tmin, tmax, npts)
    x = x0 + ap * np.cos(t) * np.cos(phi) - bp * np.sin(t) * np.sin(phi)
    y = y0 + ap * np.cos(t) * np.sin(phi) + bp * np.sin(t) * np.cos(phi)
    return x, y
    
x = pc_space_points[:,0]
y = pc_space_points[:,1]

coeffs = fit_ellipse(x, y)
print('Fitted parameters:')
print('a, b, c, d, e, f =', coeffs)
x0, y0, ap, bp, e, phi = cart_to_pol(coeffs)
print('x0, y0, ap, bp, e, phi = ', x0, y0, ap, bp, e, phi)

plt.scatter(x, y)     # given points
x, y = get_ellipse_pts((x0, y0, ap, bp, e, phi))
plt.plot(x, y)
plt.show()

In [ ]:
def find_closest_point(data_point, ellipse_center, semi_major_axis, semi_minor_axis, rotation_angle):
    # Step 1: Define the ellipse
    h, k = ellipse_center
    a = semi_major_axis
    b = semi_minor_axis
    theta = math.radians(rotation_angle)
    
    # Step 2: Translate the coordinates
    translated_data_point = (data_point[0] - h, data_point[1] - k)
    
    # Step 3: Apply rotation
    x, y = translated_data_point
    cos_theta = math.cos(-theta)
    sin_theta = math.sin(-theta)
    rotated_data_point = (
        x * cos_theta - y * sin_theta,
        x * sin_theta + y * cos_theta
    )
    
    # Step 4: Calculate the closest point on the ellipse
    x_norm = rotated_data_point[0] / a
    y_norm = rotated_data_point[1] / b
    distance = math.sqrt(x_norm ** 2 + y_norm ** 2)
    alpha = math.atan2(y_norm, x_norm)
    closest_point_norm = (a * math.cos(alpha), b * math.sin(alpha))
    
    # Step 5: Apply inverse rotation and translation
    x_closest_norm, y_closest_norm = closest_point_norm
    x_closest = x_closest_norm * cos_theta + y_closest_norm * sin_theta
    y_closest = -x_closest_norm * sin_theta + y_closest_norm * cos_theta
    closest_point = (x_closest + h, y_closest + k)
    
    return closest_point

def get_projected_angle(data_point, ellipse_center, semi_major_axis, semi_minor_axis, rotation_angle):
    # Step 1: Define the ellipse
    h, k = ellipse_center
    a = semi_major_axis
    b = semi_minor_axis
    theta = math.radians(rotation_angle)
    
    # Step 2: Translate the coordinates
    translated_data_point = (data_point[0] - h, data_point[1] - k)
    
    # Step 3: Apply rotation
    x, y = translated_data_point
    cos_theta = math.cos(-theta)
    sin_theta = math.sin(-theta)
    rotated_data_point = (
        x * cos_theta - y * sin_theta,
        x * sin_theta + y * cos_theta
    )
    
    # Step 4: Calculate the angle of the projected data point
    x_norm = rotated_data_point[0] / a
    y_norm = rotated_data_point[1] / b
    alpha = math.degrees(math.atan2(y_norm, x_norm))
    beta = alpha - rotation_angle
    
    return beta


def get_direction_prediction(datapoint, x0, y0, ap, bp, phi):
    closest_point = find_closest_point(datapoint, (x0, y0), ap, bp, phi)
    projected_angle = get_projected_angle(datapoint, (x0, y0), ap, bp, phi)
    
    if projected_angle < 0: projected_angle = 360 + projected_angle
    
    sector_length = 360 / 8
    
    for i in range(8):
        if projected_angle >= i * sector_length and projected_angle < (i + 1) * sector_length: 
            return i + 1

datapoint = (3000, -500)
# get_direction_prediction(datapoint,pc_space_points)

In [ ]:
# # Compute PCA from selected (train) subset of points using the direction-averaged matrix

# R2 = R.copy()
# TRAIN_TRIAL_SUBSET = (0,100)
# TEST_TRIAL_SUBSET = (100,158)
# train_R2 = R2[:, TRAIN_TRIAL_SUBSET[0]:TRAIN_TRIAL_SUBSET[1]].copy()
# test_R2 = R2[:, TEST_TRIAL_SUBSET[0]:TEST_TRIAL_SUBSET[1]].copy()

# for i in range(train_R2.shape[0]):
#     test_R2[i,:] = test_R2[i,:] - train_R2[i,:].mean() 
#     train_R2[i,:] = train_R2[i,:] - train_R2[i,:].mean() 

# pca3 = decomposition.PCA(n_components=2)
# x_pca3 = pca3.fit_transform(train_R2)

# train_pc_space_points = np.dot(train_R2.T, x_pca3[:,:])
# test_pc_space_points = np.dot(test_R2.T, x_pca3[:,:])
# # train_pc_space_points = np.flip(train_pc_space_points, axis=0)
# # test_pc_space_points = np.flip(test_pc_space_points, axis=0)
# plt.scatter(train_pc_space_points[:,0], train_pc_space_points[:,1])
# plt.scatter(test_pc_space_points[:,0], test_pc_space_points[:,1])
# x, y = get_ellipse_pts((x0, y0, ap, bp, e, phi))
# plt.plot(x, y)

# x = train_pc_space_points[:,0]
# y = train_pc_space_points[:,1]

# # (x0, y0) is the ellipse centre; 
# # (ap, bp) are the semi-major and semi-minor axes, respectively; 
# # e is the eccentricity; 
# # phi is the rotation of the semi-major axis from the x-axis
# x0, y0, ap, bp, e, phi = cart_to_pol(fit_ellipse(x, y))
# print('Fitted ellipse parameters:')
# print(x0, y0, ap, bp, e, phi)

# phi = phi + 180

# correct = 0
# for i in range(TEST_TRIAL_SUBSET[1]-TEST_TRIAL_SUBSET[0]):
#     new_x = test_pc_space_points[i,0]
#     new_y = test_pc_space_points[i,1]
#     predicted_direction = get_direction_prediction((new_x, new_y), x0, y0, ap, bp, phi)
#     true_direction = direction[TEST_TRIAL_SUBSET[0]+i,0]
#     print(f'Predicted {predicted_direction} True {true_direction}')
    
#     if true_direction==predicted_direction or true_direction==predicted_direction-1 or true_direction == predicted_direction+1: correct+=1
# print(f'Correct = {correct}')


In [ ]:
# Trying to get equidistant sections of ellipse

In [ ]:
import math

def calculate_ellipse_points(a, b, h, k, rotation_angle):
    # Calculate eight equidistant points along the ellipse
    points = []
    for i in range(8):
        theta = i * (2 * math.pi / 8)  # Angle from major axis
        # x = h + a * math.cos(theta)  # x-coordinate
        # y = k + b * math.sin(theta)  # y-coordinate
        x = h + a * math.cos(theta) * math.cos(rotation_angle) - b * math.sin(theta) * math.sin(rotation_angle)  # x-coordinate
        y = k + a * math.cos(theta) * math.sin(rotation_angle) + b * math.sin(theta) * math.cos(rotation_angle)  # y-coordinate
        points.append([x, y])

    return points

# Example usage
a = ap  # Major axis length
b = bp  # Minor axis length
h = x0  # x-coordinate of the center
k = y0  # y-coordinate of the center

ellipse_points = np.array(calculate_ellipse_points(a, b, h, k, phi))
for i, point in enumerate(ellipse_points):
    print("Point", i+1, ": (", point[0], ",", point[1], ")")


In [ ]:
x, y = get_ellipse_pts((x0, y0, ap, bp, e, phi))
plt.plot(x, y)
plt.scatter(ellipse_points[:,0], ellipse_points[:,1])

In [ ]:
get_ellipse_pts((x0, y0, ap, bp, e, phi))